In [1]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy
from keras.regularizers import l2

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATA_DIR = 'data'
HDF5_FILE = 'data_tf.hdf5'

In [4]:
BATCH_SIZE = 64
IMG_SHAPE = (96, 96, 4)

In [5]:
data_file = os.path.join(DATA_DIR, HDF5_FILE)
ds_x = tfio.IODataset.from_hdf5(data_file, '/x_train')
ds_y = tfio.IODataset.from_hdf5(data_file, '/y_train')

In [6]:
# convert to float and scale to [0, 1)
ds_x = ds_x.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
# ds_x = ds_images.map(lambda x: tf.image.per_image_standardization(x))

In [7]:
ds_xy = tf.data.Dataset.zip((ds_x, ds_y))
ds_xy = ds_xy.shuffle(BATCH_SIZE*2)

ds_len = ds_xy.reduce(0, lambda x, _: x + 1).numpy()
train_len = int(0.9 * ds_len)

ds_train = ds_xy.take(train_len)
ds_test = ds_xy.skip(train_len)

ds_len, train_len

(220025, 198022)

In [8]:
model = Sequential()

model.add(Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(2e-5))) #, input_shape=(X_train.shape[1:])))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(2e-5)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(32, (3,3), activation='relu', kernel_regularizer=l2(2e-5)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), activation='relu', kernel_regularizer=l2(2e-5)))
model.add(BatchNormalization())
model.add(MaxPooling2D (pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(16, (3,3), activation='relu', kernel_regularizer=l2(2e-5)))
model.add(BatchNormalization())
model.add(Conv2D(16, (3,3), activation='relu', kernel_regularizer=l2(2e-5)))
model.add(BatchNormalization())
model.add(MaxPooling2D (pool_size=(2, 2)))
# model.add(Dropout(0.4))
model.add(Dropout(0.5))

model.add(Flatten())
# model.add(Dropout(0.2))

# model.add(Dense(64, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.4))

model.add(Dense(1, activation='sigmoid'))

# model.summary()

In [9]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
3095/3095 - 186s - loss: 0.3898 - accuracy: 0.8391 - val_loss: 0.3934 - val_accuracy: 0.8401
Epoch 2/10
3095/3095 - 181s - loss: 0.2921 - accuracy: 0.8834 - val_loss: 0.8501 - val_accuracy: 0.7614
Epoch 3/10
3095/3095 - 181s - loss: 0.2545 - accuracy: 0.9022 - val_loss: 0.4145 - val_accuracy: 0.8522
Epoch 4/10
3095/3095 - 181s - loss: 0.2357 - accuracy: 0.9117 - val_loss: 0.3859 - val_accuracy: 0.8536
Epoch 5/10
3095/3095 - 181s - loss: 0.2246 - accuracy: 0.9171 - val_loss: 0.3678 - val_accuracy: 0.8670
Epoch 6/10
3095/3095 - 181s - loss: 0.2185 - accuracy: 0.9207 - val_loss: 0.2908 - val_accuracy: 0.8861
Epoch 7/10
3095/3095 - 181s - loss: 0.2137 - accuracy: 0.9234 - val_loss: 0.2820 - val_accuracy: 0.8973
Epoch 8/10
3095/3095 - 181s - loss: 0.2091 - accuracy: 0.9258 - val_loss: 0.2974 - val_accuracy: 0.8847
Epoch 9/10
3095/3095 - 181s - loss: 0.2065 - accuracy: 0.9270 - val_loss: 0.8660 - val_accuracy: 0.7813
Epoch 10/10
3095/3095 - 181s - loss: 0.2024 - accuracy: 0.9296 -

In [10]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=2e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
1548/1548 - 183s - loss: 0.1725 - accuracy: 0.9421 - val_loss: 0.1749 - val_accuracy: 0.9403
Epoch 2/10
1548/1548 - 181s - loss: 0.1672 - accuracy: 0.9442 - val_loss: 0.1858 - val_accuracy: 0.9394
Epoch 3/10
1548/1548 - 189s - loss: 0.1658 - accuracy: 0.9447 - val_loss: 0.1660 - val_accuracy: 0.9458
Epoch 4/10
1548/1548 - 190s - loss: 0.1635 - accuracy: 0.9458 - val_loss: 0.1604 - val_accuracy: 0.9446
Epoch 5/10
1548/1548 - 190s - loss: 0.1622 - accuracy: 0.9462 - val_loss: 0.1550 - val_accuracy: 0.9502
Epoch 6/10
1548/1548 - 189s - loss: 0.1613 - accuracy: 0.9462 - val_loss: 0.1628 - val_accuracy: 0.9456
Epoch 7/10
1548/1548 - 180s - loss: 0.1586 - accuracy: 0.9472 - val_loss: 0.1687 - val_accuracy: 0.9443
Epoch 8/10
1548/1548 - 190s - loss: 0.1580 - accuracy: 0.9474 - val_loss: 0.1627 - val_accuracy: 0.9472
Epoch 9/10
1548/1548 - 190s - loss: 0.1573 - accuracy: 0.9476 - val_loss: 0.1735 - val_accuracy: 0.9427
Epoch 10/10
1548/1548 - 190s - loss: 0.1558 - accuracy: 0.9484 -

In [11]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
774/774 - 191s - loss: 0.1481 - accuracy: 0.9517 - val_loss: 0.1475 - val_accuracy: 0.9531
Epoch 2/10
774/774 - 194s - loss: 0.1474 - accuracy: 0.9517 - val_loss: 0.1460 - val_accuracy: 0.9539
Epoch 3/10
774/774 - 200s - loss: 0.1466 - accuracy: 0.9520 - val_loss: 0.1474 - val_accuracy: 0.9530
Epoch 4/10
774/774 - 199s - loss: 0.1466 - accuracy: 0.9519 - val_loss: 0.1465 - val_accuracy: 0.9533
Epoch 5/10
774/774 - 197s - loss: 0.1463 - accuracy: 0.9522 - val_loss: 0.1496 - val_accuracy: 0.9521
Epoch 6/10
774/774 - 188s - loss: 0.1453 - accuracy: 0.9526 - val_loss: 0.1488 - val_accuracy: 0.9525
Epoch 7/10
774/774 - 183s - loss: 0.1449 - accuracy: 0.9528 - val_loss: 0.1556 - val_accuracy: 0.9507
Epoch 8/10
774/774 - 195s - loss: 0.1448 - accuracy: 0.9525 - val_loss: 0.1513 - val_accuracy: 0.9514
Epoch 9/10
774/774 - 189s - loss: 0.1450 - accuracy: 0.9524 - val_loss: 0.1461 - val_accuracy: 0.9533
Epoch 10/10
774/774 - 200s - loss: 0.1446 - accuracy: 0.9529 - val_loss: 0.1476 - 

In [13]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
774/774 - 200s - loss: 0.1440 - accuracy: 0.9526 - val_loss: 0.1432 - val_accuracy: 0.9546
Epoch 2/10
774/774 - 198s - loss: 0.1434 - accuracy: 0.9532 - val_loss: 0.1442 - val_accuracy: 0.9534
Epoch 3/10
774/774 - 198s - loss: 0.1433 - accuracy: 0.9532 - val_loss: 0.1444 - val_accuracy: 0.9538
Epoch 4/10
774/774 - 198s - loss: 0.1435 - accuracy: 0.9526 - val_loss: 0.1440 - val_accuracy: 0.9535
Epoch 5/10
774/774 - 200s - loss: 0.1431 - accuracy: 0.9534 - val_loss: 0.1441 - val_accuracy: 0.9533
Epoch 6/10
774/774 - 190s - loss: 0.1427 - accuracy: 0.9534 - val_loss: 0.1438 - val_accuracy: 0.9535
Epoch 7/10
774/774 - 198s - loss: 0.1431 - accuracy: 0.9539 - val_loss: 0.1432 - val_accuracy: 0.9536
Epoch 8/10
774/774 - 187s - loss: 0.1431 - accuracy: 0.9532 - val_loss: 0.1433 - val_accuracy: 0.9540
Epoch 9/10
774/774 - 191s - loss: 0.1418 - accuracy: 0.9540 - val_loss: 0.1433 - val_accuracy: 0.9539
Epoch 10/10
774/774 - 197s - loss: 0.1434 - accuracy: 0.9529 - val_loss: 0.1431 - 